In [119]:
import pickle
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
ACTIONS = ['UP', 'RIGHT', 'DOWN', 'LEFT', 'WAIT', 'BOMB']
ACTION_TO_INT = {'UP':0, 'RIGHT':1, 'DOWN':2, 'LEFT':3, 'WAIT':4, 'BOMB':5}

In [120]:
# TODO: check if the linear value approximation actually works, by approximating only for one case
# TODO: check out regression forests
# TODO: check put PyTables to save and load entries of large dictionaries saved as a file

In [121]:
with open("my-saved-q-table.pt", "rb") as file:
    q_table:dict = dict(pickle.load(file))
with open("action_vectors.pt", "rb") as file:
    action_vectors:dict = dict(pickle.load(file))

In [122]:
def state_to_action_via_lin_val_est(state_str:str):
    temp_str = state_str[1:-1]
    state = np.fromstring(temp_str, dtype=float, sep=' ')
    Q = np.zeros(len(ACTIONS))
    for i, a in enumerate(ACTIONS):
        Q[i] = np.dot(state, action_vectors[a])
    action_int = Q.argmax()
    return ACTIONS[action_int]

In [123]:
c = 0
for key in sorted(q_table.keys()):
    #print(key)
    action = q_table[key].argmax()
    print(key, "simple_q_learning: ", ACTIONS[action], "lin_val_approx: ", state_to_action_via_lin_val_est(key) )
    if (ACTIONS[action] ==state_to_action_via_lin_val_est(key)):
        c += 1
    print(q_table[key] )
    print()

[ 0  1  1  1  1 -1] simple_q_learning:  WAIT lin_val_approx:  RIGHT
[-278.89412561 -312.55907192 -281.12689843 -282.73274067 -257.23282127
 -327.47470478]

[ 0  1  1  2  1 -1] simple_q_learning:  RIGHT lin_val_approx:  RIGHT
[-132.86555254  -16.84146976 -175.60121646 -154.67981192 -209.94486368
 -182.36837033]

[ 0  1  1  3  1 -1] simple_q_learning:  RIGHT lin_val_approx:  RIGHT
[-219.78503348  -35.17413658 -184.96390878 -218.38540244 -153.45760031
 -216.18108294]

[ 0  1  1 -1  1 -1] simple_q_learning:  RIGHT lin_val_approx:  RIGHT
[-266.64064665 -159.35224611 -273.71038091 -319.89958317 -210.40402823
 -192.22915748]

[ 0  1  1 -2  1 -1] simple_q_learning:  LEFT lin_val_approx:  RIGHT
[-187.44433159 -156.02144904 -122.39664621 -115.33417232 -178.19782478
 -168.90483773]

[ 0  1  2  1  1 -1] simple_q_learning:  RIGHT lin_val_approx:  RIGHT
[-335.63738304 -125.41614735 -284.91581895 -319.79820372 -316.63135292
 -345.83812413]

[ 0  1  2  2  1 -1] simple_q_learning:  RIGHT lin_val_approx

In [124]:
print(len(q_table)) 
print(c)

66
37


In [125]:
X = np.eye(3)   # <- samples / features
#X = X.reshape(-1, 1)
y = np.array([1, 2, 3]) # <- tags / Q-val
X

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [126]:
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X,y)
y_pred = regressor.predict([[0, 0, 1]])
y_pred



array([3.])

In [129]:
regressor = GradientBoostingRegressor(random_state = 0)
regressor.fit(X,y)
y_pred = regressor.predict([[1, 0, 0]])
y_pred

array([1.00002656])